In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers

def make_generator_model(width, height):
    model = tf.keras.Sequential()

    # Adjust the Dense layer's units to match the target size
    model.add(layers.Dense((width // 4) * (height // 4) * 256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((width // 4, height // 4, 256)))

    # Upscale to half of the width and height
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upscale to the full width and height
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

def make_discriminator_model(width, height):
    model = tf.keras.Sequential()

    # Adjust the input shape to match the size of your images
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[width, height, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Further downscale
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Flatten and output a single value for real/fake classification
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

# Create the generator and discriminator models
generator = make_generator_model(width, height)
discriminator = make_discriminator_model(width, height)

# Define the loss function and the optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Saving checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
import cv2
import os
import numpy as np

directory_path = 'MCI/video_sequence'
frame_list = []

for filename in os.listdir(directory_path):
    if filename.endswith(".avi"):
        video_path = os.path.join(directory_path, filename)
        cap = cv2.VideoCapture(video_path)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Resize the frame
            resized_frame = cv2.resize(gray_frame, (width, height))

            # Normalize the frame to [-1, 1]
            normalized_frame = (resized_frame - 127.5) / 127.5

            # Add the frame to the list
            frame_list.append(normalized_frame)

        cap.release()

cv2.destroyAllWindows()

# Convert list to NumPy array
frame_array = np.array(frame_list)

# Shuffle the data
np.random.shuffle(frame_array)

# Checking the shape and range of the frames
print("Number of frames:", len(frame_array))
print("Shape of a frame:", frame_array[0].shape)
print("Data type:", frame_array.dtype)
print("Min and Max pixel values:", frame_array.min(), frame_array.max())

# Batch the data
BUFFER_SIZE = 60000  # Adjust this based on your dataset size
BATCH_SIZE = 256  # Adjust based on your GPU capacity and training requirements

# Batch and shuffle the data with TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices(frame_array).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)


In [ ]:
import tensorflow as tf
import time

EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime to visualize progress in the animated GIF
seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
    start_time = time.time()

    for epoch in range(epochs):
        epoch_start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        epoch_end = time.time()
        epoch_time = epoch_end - epoch_start
        total_time = epoch_end - start_time
        avg_time_per_epoch = total_time / (epoch + 1)
        remaining_epochs = epochs - epoch - 1
        eta = avg_time_per_epoch * remaining_epochs

        print(f'Epoch {epoch}/{epochs} completed in {epoch_time:.2f} sec. ETA: {eta:.2f} sec.')

# Train the GAN
train(train_dataset, EPOCHS)

In [ ]:
from matplotlib import pyplot as plt

def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Generate after the final epoch
generate_and_save_images(generator, EPOCHS, seed)
